# Web Crawler for NDHU Campus Reservation System

<h2>Imports & SetUps</h2>

In [63]:
%pip install beautifulsoup4
import requests
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [73]:
from bs4 import BeautifulSoup

#testCase 1
url = 'https://sys.ndhu.edu.tw/SA/XSL_ApplyRWD/ActApply.aspx'
html = requests.get(url)
html.encoding="utf-8"
sp = BeautifulSoup(html.text, 'html.parser')
# ensure get the english version of the webpage
print(sp.prettify())



<!DOCTYPE html>
<html lang="zh-hant">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   跨域自主學習活動暨報名系統
  </title>
  <link href="favicon.ico" rel="shortcut icon"/>
  <link href="Content/masterpage.min.css" rel="stylesheet"/>
  <style type="text/css">
   /*.navbar-default{background-color:#408eba;border-color:#3a80a7}  原先masterpage.min.css 中定義的藍色系 */
        .navbar-default {
            background-color:#389848;  /* banner, 綠色系 */
            border-color:#2B7837;
        }

        /* .navbar-default .navbar-toggle{border-color:#3a80a7}            行動版選單的外框顏色
           .navbar-default .navbar-toggle:focus{background-color:#3a80a7}  行動版選單在點選後的顏色
           .navbar-default .navbar-toggle .icon-bar{background-color:#888} 行動版選單 上的三條線顏色
        */
        .navbar-default .navbar-toggle {  border-color:#2B7837;  }
        /*.navbar-default .navbar-toggle:focus {  backg

In [72]:
#testCase 2 (Cannot obtain certified hours)
# url = 'https://sys.ndhu.edu.tw/SA/XSL_ApplyRWD/ActApply.aspx?lang=en'
# html1 = requests.get(url)
sp = BeautifulSoup(html, 'html.parser')

print(sp.title)

TypeError: object of type 'Response' has no len()

In [74]:
#find the table located on the webpage that consists of ten activities per page
reserv_tb= sp.find(id="BodyContent_gvActs", class_ = "grid table table-striped table-bordered table-hover")

print(reserv_tb.prettify())

<table border="1" cellspacing="0" class="grid table table-striped table-bordered table-hover" id="BodyContent_gvActs" rules="all" style="margin-top: 15px;">
 <tr class="info">
  <th scope="col" width="100">
  </th>
  <th scope="col" width="48">
   <font size="3">
    我的
    <br/>
    收藏
   </font>
  </th>
  <th scope="col" width="70">
   <a href="javascript:__doPostBack('ctl00$BodyContent$gvActs$ctl01$lbt_sort_status','')" id="BodyContent_gvActs_lbt_sort_status">
    狀態
   </a>
  </th>
  <th scope="col" width="84">
   <a href="javascript:__doPostBack('ctl00$BodyContent$gvActs$ctl01$lbt_sort_xsl_check','')" id="BodyContent_gvActs_lbt_sort_xsl_check">
    能否認證/認證時數
   </a>
  </th>
  <th scope="col" width="90">
   參加對象
  </th>
  <th scope="col">
   <a href="javascript:__doPostBack('ctl00$BodyContent$gvActs$ctl01$lbt_sort_act_name','')" id="BodyContent_gvActs_lbt_sort_act_name">
    活動名稱
   </a>
  </th>
  <th scope="col" width="162">
   <!-- 報名日期時間 20220803改為 截止報名時間 -->
   <a href="javascr

<h2>Extracting Info from the WebPage</h2>

In [75]:
# Formatting & Cleaning the rows of the Table

reservtb_rows = reserv_tb.find_all('tr')

# print(reservtb_rows[0].prettify())

# remove all non-activity rows (first,last,waiting list tables and pagination row)
preserveList = []
for row in reservtb_rows:
      rowQuery = row.find("td", align="right")
      if rowQuery != None:
            preserveList.append(row)
      del rowQuery

for row in preserveList:
        print(row.prettify())
        print("----Entry----")



<tr>
 <td align="right">
  <a class="btn btn-primary btn-sm" href="javascript:__doPostBack('ctl00$BodyContent$gvActs$ctl02$lbtGridApply','')" id="BodyContent_gvActs_lbtGridApply_0" style="margin-top:3px;">
   報名
  </a>
  <a class="btn btn-primary btn-sm" href="javascript:__doPostBack('ctl00$BodyContent$gvActs$ctl02$lbtGridShow','')" id="BodyContent_gvActs_lbtGridShow_0" style="margin-top:3px;">
   檢視
  </a>
 </td>
 <td>
  <strong class="visible-xs-inline">
   我的收藏
  </strong>
  <a href="javascript:__doPostBack('ctl00$BodyContent$gvActs$ctl02$lbtCollectAdd','')" id="BodyContent_gvActs_lbtCollectAdd_0" title="加入收藏">
   <font color="Blue">
    <span class="glyphicon glyphicon-plus-sign">
    </span>
   </font>
  </a>
 </td>
 <td>
  <strong class="visible-xs-inline">
   <!-- 要有這行，手機版才顯示 -->
   狀態
  </strong>
  <br class="visible-lg"/>
  <span id="BodyContent_gvActs_lblGv_status_0">
   報名中
  </span>
 </td>
 <td>
  <strong class="visible-xs-inline">
   能否認證/認證時數
  </strong>
  <span id="BodyC

In [79]:
import re

activityList = []

for actNum, row in enumerate(preserveList):
    activity = []

    # activity name
    act_name = row.find("span", id="BodyContent_gvActs_lblGv_act_name_"+str(actNum)).get_text()
    act_name=act_name.strip()
    activity.append(act_name)
    

    # certified number of hours
    certHours_str = row.find("span", id="BodyContent_gvActs_lblGv_xsl_check_"+str(actNum)).get_text()

    # print(act_name,certHours_str)

    print(certHours_str)

    certHours_num = 0 
    if certHours_str == "Cannot":
        print("cannot")
        activity.append(certHours_num)
    else: 
        certHours_num = re.findall(r'[\d]*[.][\d]+|[\d]+', certHours_str)
        activity.append(certHours_num[0])
        

    # number of participants (primary num/num)
    num_registered_str = row.find("span", id="BodyContent_gvActs_lblGv_reg_num_"+str(actNum)).get_text()
    # only already registered students
    num_registered_num = num_registered_str.split("/", 1)[0]
    num_registered_num = num_registered_num.strip()
    activity.append(num_registered_num)

    # registration deadline
    registration_deadline = row.find("span", id="BodyContent_gvActs_lblGv_reg_dt_"+str(actNum)).get_text()
    registration_deadline  = registration_deadline.strip()
    activity.append(registration_deadline)

   # activity date
    activity_date = row.find("span", id="BodyContent_gvActs_lblGv_act_dt_"+str(actNum)).get_text()
    activity_date  = activity_date.strip()
    activity.append(activity_date)

    # print(activity)
    

    activityList.append(activity)

    del activity


可 / 2
可 / 3
可 / 2
可 / 3
可 / 3
可 / 1
可 / 2
可 / 2
可 / 2
可 / 2


# Pandas DataFrame

In [77]:
# for activity_item in activityList:
#     print(activity_item)

In [80]:
output_tb = pd.DataFrame(activityList,columns=['Name Of Activty','Number of Hours Certified','Number Of People Who Registered','Date and Time of Registration','Date and Time of The Activity'])

display(output_tb)

,Name Of Activty,Number of Hours Certified,Number Of People Who Registered,Date and Time of Registration,Date and Time of The Activity
0,【會計/企管/資管】校友回娘家,2,59,2023/05/24(三)00:00~ 2023/06/02(五)10:00,2023/06/02(五) 12:20~14:00
1,113級教保實習成果展《幼來上課囉！》,3,50,2023/05/25(四)00:00~ 2023/06/02(五)12:00,2023/06/04(日) 09:00~12:00
2,2023東華春藝季~三角關係鋼琴音樂會-「絕代風華」,2,78,2023/05/24(三)00:00~ 2023/06/02(五)18:00,2023/06/02(五) 19:00~21:00
3,【111-2 用影像關注性別議題】成果展,3,16,2023/05/25(四)00:00~ 2023/06/03(六)12:00,2023/06/04(日) 13:00~16:00
4,看看你幹了什麼好事——學期經驗統整單次團體,3,8,2023/05/25(四)00:00~ 2023/06/05(一)00:00,2023/06/07(三) 18:00~21:00
5,【藝創系】112.6.6『111-2藝創系系學會期末大會』(認證1小時，限本系學生),1,60,2023/05/30(二)19:00~ 2023/06/06(二)12:00,2023/06/06(二) 18:00~19:00
6,MISSION IMPOSSIBLE XXXII ～鋼琴獨奏、四手聯彈～ 林世悠師生鋼琴演奏...,2,18,2023/05/19(五)00:00~ 2023/06/07(三)00:00,2023/06/09(五) 08:30~10:00
7,MISSION IMPOSSIBLE XXXIII ～鋼琴獨奏、四手聯彈～ 林世悠師生鋼琴演...,2,25,2023/05/19(五)00:00~ 2023/06/07(三)00:00,2023/06/09(五) 10:30~12:00
8,【光電系系學會】期末供品【限系學會工作人員報名】,2,5,2023/05/30(二)00:00~ 2023/06/08(四)00:00,2023/06/09(五) 18:30~20:30
9,Friend or Foe: Navigating Geopolitical Risk,2,34,2023/05/29(一)12:00~ 2023/06/08(四)12:00,2023/06/09(五) 12:10~14:00
